# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1
# Create table 1

> We want to retrieve data by session_id and item_in_session.
> The partition key will be session_id and item_in_session will be the clustering colums. 
> These two fields together are a unique key for the table.

In [ ]:
## Query 1: Give me only the following: name of artist, song (sorted by itemInSession) and song's lenght \
## for sessionid = 338 and itemInSession = 4
music_table = "artist_song_by_session"

create_music_table = "CREATE TABLE IF NOT EXISTS {0}".format(music_table)
create_music_table = create_music_table + "(session_id int, item_in_session int, artist text, first_name text, gender text, last_name text, length double, level text, location text, song text, user_id int, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(create_music_table)
except Exception as e:
    print(e)
                    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO {0} (session_id, item_in_session, artist, length, song)".format(music_table)
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
query1 = "select artist, song, length from {0} where session_id=338 and item_in_session=4".format(music_table)

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, " | ",row.song, " | ",row.length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### QUERY 2
# Create table 2

> We want to retrieve data by user_id and session_id, but we want also data orderred by item_in_session.
> The partition key will be user_id, session_id, and item_in_session will be the clustering colums. 
> So user can use session_id in WHERE condition and the result will be sorted as requested.

In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

music_table2 = "artist_song_user_by_user_session"

create_music_table2 = "CREATE TABLE IF NOT EXISTS {0}".format(music_table2)
create_music_table2 = create_music_table2 + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(create_music_table2)
except Exception as e:
    print(e)               

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO {0} (user_id, session_id, item_in_session, artist, song, first_name, last_name)".format(music_table2)
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
query2= "select artist, song, first_name, last_name from {0} where user_id=10 and session_id=182".format(music_table2)

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, " | ", row.song, " | ", row.first_name, " | ", row.last_name)
    print("_____________________________________________________________________________________")

### QUERY 3
# Create table 3
> We want to select the user data using the song title.
> The partition key must be the song field, but it will be not unique. I chose to use the fields song and user_id to create a unique primary key. 
> It's enough for this query, because all I want to know is if a user listened a song.

In [ ]:
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

music_table3 = "user_by_song"

create_music_table3 = "CREATE TABLE IF NOT EXISTS {0}".format(music_table3)
create_music_table3 = create_music_table3 + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))"
try:
    session.execute(create_music_table3)
except Exception as e:
    print(e)    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO {0} (song, user_id, first_name, last_name)".format(music_table3)
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
query3= "select first_name, last_name from {0} where song='All Hands Against His Own'".format(music_table3)

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, " | ", row.last_name)
    print("_____________________________________________________________________________________")

### Drop the tables before closing out the sessions

In [ ]:
query = f"drop table if exists {music_table}"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = f"drop table if exists {music_table2}"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = f"drop table if exists {music_table3}"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()